In [22]:
import pickle
import nltk
import json
from tqdm import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer

APP_FOLDER = '../app/data/processed/'

In [23]:
with open('./categories/Chinese.pkl', 'rb') as f:
    cuisine_reviews = pickle.load(f)    
with open('ChineseDishes.txt', 'r') as f:
    dish_names = [line.replace("\n", "").lower() for line in f.readlines()]

for _review in tqdm(cuisine_reviews):
    _review['sentences'] = nltk.sent_tokenize(_review['text'])

sia = SentimentIntensityAnalyzer()

dish2rest = {}
for _name in tqdm(dish_names):
    _review_count = 0
    _sum_review_polarity = 0
    _sum_rest_polarity = 0
    
    for _review in cuisine_reviews:
        _rest_id = _review['business_id']
        _review_id = _review['review_id']
        _review_polarity = 0
        _review_sentence_count = 0
        for _sentence in _review['sentences']:
            _count = _sentence.count(_name)
            if _count > 0:
                _review_sentence_count += 1
                _ss = sia.polarity_scores(_sentence)
                _score = (_ss['pos'] + 1e-10)/ (_ss['neg'] + 1e-10) * 0.5
                if _score <= 1:
                    _review_polarity += _score
                else:
                    _review_polarity = 1
        
        if _review_sentence_count > 0:
            _review_polarity = _review_polarity / _review_sentence_count
            _sum_review_polarity += _review_polarity
            _review_count += 1
            if not(_name in dish2rest):
                dish2rest[_name] = {'rest2review_count':{}, 
                                     'review2polarity':{}, 
                                     'rest2polarity':{}, 
                                     'rest2score':{},
                                     'review_count': 0,
                                     'rest_count': 0}
                    
            dish2rest[_name]['review2polarity'][_review_id] = _review_polarity
            if _rest_id in dish2rest[_name]['rest2review_count']:
                dish2rest[_name]['rest2review_count'][_rest_id] += 1
                dish2rest[_name]['rest2polarity'][_rest_id] += _review_polarity
            else:
                dish2rest[_name]['rest2review_count'][_rest_id] = 1
                dish2rest[_name]['rest2polarity'][_rest_id] = _review_polarity
            
    if _name in dish2rest:
        for _id in dish2rest[_name]['rest2polarity']:
            dish2rest[_name]['rest2score'][_id] = dish2rest[_name]['rest2polarity'][_id]
            dish2rest[_name]['rest2polarity'][_id] /= dish2rest[_name]['rest2review_count'][_id]
            _sum_rest_polarity += dish2rest[_name]['rest2polarity'][_id]

        dish2rest[_name]['rest2polarity'] = sorted(dish2rest[_name]['rest2polarity'].items(), key=lambda kv: kv[1], reverse = True)
        dish2rest[_name]['rest2review_count'] = sorted(dish2rest[_name]['rest2review_count'].items(), key=lambda kv: kv[1], reverse = True)
        dish2rest[_name]['rest2score'] = sorted(dish2rest[_name]['rest2score'].items(), key=lambda kv: kv[1], reverse = True)
        dish2rest[_name]['review_count'] = len(dish2rest[_name]['review2polarity'])
        dish2rest[_name]['rest_count'] = len(dish2rest[_name]['rest2review_count'])
        dish2rest[_name]['rest_polarity'] = _sum_rest_polarity / dish2rest[_name]['rest_count']
        dish2rest[_name]['review_polarity'] = _sum_review_polarity / dish2rest[_name]['review_count']

sorted_rest_dishes = sorted(dish2rest.items(), key=lambda kv: kv[1]['rest_count'], reverse=True)
sorted_review_dishes = sorted(dish2rest.items(), key=lambda kv: kv[1]['review_count'], reverse=True)

with open (APP_FOLDER + 'dish2rest.plk', 'wb') as f:
    pickle.dump(sorted_rest_dishes, f)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 193/193 [00:38<00:00,  6.57it/s]


In [24]:
with open (APP_FOLDER + 'restinfo.plk', 'rb') as f:
    rest_info = pickle.load(f)

In [25]:
rest_info['wUPHVWGmoBU_gykHl1Kzxg']['review_summary_gensim']

"With that said, in comparison to the various Chinese places in the Tempe/Scottsdale area, the quality and taste of the food here is pretty good for the price.\nI know it's been around a while - maybe they are regulars.I live nearby and have been to this restaurant many times.The food - good - always tasty and I can recommend the sweet and sour soup, tofu vegetables, pork and vegetables, beef and broccoli - never had a bad dish here.The service - better to do take-out than dine-in."

In [26]:
rest_info['wUPHVWGmoBU_gykHl1Kzxg']['review_summary_pytease']

["And it's served in a warm cozy setting by friendly staff at a reasonable price.I like Chinese food.",
 'Now you know where my addiction started.Went back again; Some of the best authentic Chinese food around.',
 "It's a restaurant with no windows and the inside isn't totally chic, but it's no-nonsense mostly Cantonese style Chinese food with a hint of Thai.",
 "Totally recommend!!I'm typically not a Chinese food eater, but my fiance loves it.",
 'A good overall choice for a casual dinner with friends and family.Joyful is a great place to go for some good Chinese food.']

In [27]:
for k in sorted_rest_dishes:
    print(k[0])

fried rice
egg roll
orange chicken
egg rolls
lunch special
chow mein
soy sauce
egg drop
white rice
hot and sour soup
lo mein
egg drop soup
spring rolls
won ton
wonton soup
kung pao
sesame chicken
deep fried
pot sticker
pot stickers
crab puffs
brown rice
mongolian beef
green onion
noodle soup
black bean
chow fun
crab rangoon
kung pao chicken
walnut shrimp
chicken wings
green beans
won ton soup
iced tea
lemon chicken
lettuce wrap
teriyaki chicken
lettuce wraps
stir fry
bbq pork
bok choy
pad thai
egg foo young
cashew chicken
spare ribs
green tea
shopping center
snow peas
string bean
general tso
chow mien
pork chop
black pepper
hot pot
beef chow fun
honey walnut shrimp
ice tea
string beans
happy hour
bean curd
general tso's
pork chops
roast duck
egg flower soup
pork belly
beef noodle soup
roasted duck
red bean
sea bass
short ribs
baby bok choy
general tso's chicken
mu shu
clay pot
roast pork
gluten free
asian cuisine
lo mien
sizzling rice soup
milk tea
grand opening
char siu
peking duck
ch